In [1]:
import corpus
import os

In [2]:
import CaboCha

In [3]:
c = CaboCha.Parser()
txt = 'そうしたら犬が走ってきた。'
#txt = 'そうしたらそのワンちゃんがなんかか喜んじゃって、で、あたしの方に走ってきて、とびついてきちゃってさ。'
tree = c.parse(txt)
print(tree.toString(CaboCha.FORMAT_LATTICE))

* 0 2D 0/0 -2.098069
そうしたら	接続詞,*,*,*,*,*,そうしたら,ソウシタラ,ソーシタラ
* 1 2D 0/1 -2.098069
犬	名詞,一般,*,*,*,*,犬,イヌ,イヌ
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
* 2 -1D 0/3 0.000000
走っ	動詞,自立,*,*,五段・ラ行,連用タ接続,走る,ハシッ,ハシッ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
き	動詞,非自立,*,*,カ変・クル,連用形,くる,キ,キ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。
EOS



In [4]:
id = 0
for i in range(tree.size()):
    token = tree.token(i)
    if(token.chunk):
        tmp = token.chunk
        print(id, tmp.link, tmp.head_pos, tmp.func_pos, token.surface)
        id += 1

0 2 0 0 そうしたら
1 2 0 1 犬
2 -1 0 3 走っ


In [5]:
def bunsetsu_list(tree):
    result = []
    chunkindex = 0
    for i in range(tree.size()):
        token = tree.token(i)
        if(token.chunk is not None):
            features = token.feature.split(',')
            result.append({'lemma':features[-3], 'link':token.chunk.link, 'index':chunkindex})
            chunkindex += 1
    return result

def search_candidate(known_word_index, blist):
    result = set()
    for b in blist:
        if((b['link'] == known_word_index) or (b['index'] == blist[known_word_index]['link'])):
            result.add(b['index'])
    return sorted(list(result))

In [6]:
cp = corpus.Corpus(os.path.join('data','nucc','data001.txt'))

確認してください: "Ｘ：１：＊＊＊。"


In [26]:
dic = dict()
with open(os.path.join('seeds', 'positive.txt')) as f:
    for w in f.readlines():
        sw = w.strip()
        dic[sw] = corpus.word(sw)
        dic[sw].set_value(1.)
with open(os.path.join('seeds', 'negative.txt')) as f:
    for w in f.readlines():
        sw = w.strip()
        dic[sw] = corpus.word(sw)
        dic[sw].set_value(-1.)


In [27]:
dic

{'喜ぶ': <corpus.word at 0x10daa9fd0>,
 '嬉しい': <corpus.word at 0x10daa9f40>,
 '楽しい': <corpus.word at 0x10daa9ee0>,
 '美味しい': <corpus.word at 0x10daa9e50>,
 'きれい': <corpus.word at 0x10daa9e80>,
 '美しい': <corpus.word at 0x10d7b7220>,
 '楽しむ': <corpus.word at 0x10d7b77f0>,
 '悲しい': <corpus.word at 0x10daa9370>,
 'さみしい': <corpus.word at 0x10d7b72e0>,
 '辛い': <corpus.word at 0x10d7b71f0>,
 '寂しい': <corpus.word at 0x10d7b7430>,
 'きたない': <corpus.word at 0x10d7b7c40>,
 '痛い': <corpus.word at 0x10d7b7610>,
 'まずい': <corpus.word at 0x10d7b77c0>,
 '残念': <corpus.word at 0x10d7b7ac0>}

In [28]:
def calc_score(cp, argdic):
    c = CaboCha.Parser()
    dic = argdic.copy()
    for line in cp.conversation:
        tree = c.parse(line['content'])
        blist = bunsetsu_list(tree)
        for bindex, b in enumerate(blist):
            try:
                dic['lemma'].accesscount += 1
                if(dic[b['lemma']].isvisited):
                    candidates = search_candidate(bindex, blist)
                    for candidate_index in candidates:
                        candidate = blist[candidate_index]
                        try:
                            dic[candidate['lemma']].add_score(corpus.extract(dic[b['lemma']], dic[candidate['lemma']]))
                        except KeyError:
                            newword = corpus.word(candidate['lemma'])
                            newword.add_score(corpus.extract(dic[b['lemma']], newword))
                            dic[candidate['lemma']] = newword
            except KeyError:
                continue
    return dic

def calc_value(argdic, alpha=0.5):
    dic = argdic.copy()
    for key in dic.keys():
        elem = dic[key]
        if(len(elem.score) < 1):
            continue
        elif(elem.isvisited):
            dic[key].set_value(sum(elem.score)/len(elem.score) * alpha + elem.value * (1-alpha))
        else:
            dic[key].set_value(sum(elem.score)/len(elem.score))
        dic[key].reset_score()
            
    return dic
        

In [38]:
for file in sorted(os.listdir(os.path.join('moddata','nucc'))):    
    try:
        cp = corpus.Corpus(os.path.join('moddata','nucc',file))
    except Exception as e:
        print(file, e)
        pass
        #print(file)
    #dic = calc_score(cp, dic)
#dic = calc_value(dic)

    

not enough values to unpack (expected 2, got 1)
not enough values to unpack (expected 2, got 1)
not enough values to unpack (expected 2, got 1)
not enough values to unpack (expected 2, got 1)
not enough values to unpack (expected 2, got 1)
not enough values to unpack (expected 2, got 1)
not enough values to unpack (expected 2, got 1)
not enough values to unpack (expected 2, got 1)
not enough values to unpack (expected 2, got 1)
not enough values to unpack (expected 2, got 1)
not enough values to unpack (expected 2, got 1)
not enough values to unpack (expected 2, got 1)
not enough values to unpack (expected 2, got 1)


In [24]:
for i in dic.keys():
    print(dic[i])

喜ぶ, 1.0, [], (0)
嬉しい, 1.0, [], (0)
楽しい, 1.0, [], (0)
美味しい, 1.0, [], (0)
きれい, 1.0, [], (0)
美しい, 1.0, [], (0)
楽しむ, 1.0, [], (0)
悲しい, -1.0, [], (0)
さみしい, -1.0, [], (0)
辛い, -1.0, [], (0)
寂しい, -1.0, [], (0)
きたない, -1.0, [], (0)
痛い, -1.0, [], (0)
まずい, -1.0, [], (0)
残念, -1.0, [], (0)
